# 2. Selección, Limpieza y Alistamineto de Datos

## Librerias

In [2]:
import pandas as pd
import gdown
import os

## Base de Datos Inicial

In [3]:
file_id = "1rGmaBuo0xymafMghsgGMRYPPFTtwVzp3"
url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, 'Saber11.csv', quiet=False)
df = pd.read_csv('Saber11.csv')
os.remove("Saber11.csv")
df.head()

Downloading...
From (original): https://drive.google.com/uc?id=1rGmaBuo0xymafMghsgGMRYPPFTtwVzp3
From (redirected): https://drive.google.com/uc?id=1rGmaBuo0xymafMghsgGMRYPPFTtwVzp3&confirm=t&uuid=b75aa6be-f09c-4328-839c-7e733dc6b986
To: c:\Users\felip\OneDrive\Desktop\Maestria Industrial\2025-1\Analitica\Proyecto 2\Proyecto-2\Saber11.csv
100%|██████████| 297M/297M [00:10<00:00, 29.4MB/s] 


,periodo,tipodocumento,documento,zona,bilingue,calendario,caracter,daneestablecimiento,danesede,codigodpto,...,computador,internet,lavadora,nivelingles,ingles,mate,sociales,ciencias,lectura,global
0,20132,CC,SB11201320520337,URBANO,N,A,TÉCNICO/ACADÉMICO,1.110010e+11,111001015776,11,...,Si,Si,Si,NaN,40.0,44.0,NaN,NaN,NaN,NaN
1,20132,TI,SB11201320545839,URBANO,N,A,ACADÉMICO,1.080010e+11,108001002835,8,...,No,No,Si,A1,43.0,28.0,NaN,NaN,NaN,NaN
2,20132,TI,SB11201320008819,URBANO,N,A,TÉCNICO/ACADÉMICO,1.730010e+11,173001000359,73,...,Si,Si,Si,A-,43.0,43.0,NaN,NaN,NaN,NaN
3,20132,TI,SB11201320512865,RURAL,N,A,ACADÉMICO,2.707020e+11,270702000120,70,...,No,No,No,A-,39.0,34.0,NaN,NaN,NaN,NaN
4,20132,TI,SB11201320037607,URBANO,N,A,ACADÉMICO,1.118500e+11,111850001576,11,...,Si,Si,No,NaN,43.0,34.0,NaN,NaN,NaN,NaN


## Datos Inexistentes

### Cantidad de datos vacios (NaN)

In [5]:
nan_dicc = {}
for i in df.columns:
    if df[i].isna().sum() >0:
        nan_dicc[i]=df[i].isna().sum()
nan_df =pd.DataFrame(list(nan_dicc.items()),columns=["Variable","NaN"])
nan_df.T

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Variable,tipodocumento,zona,bilingue,caracter,daneestablecimiento,sedeprincipal,coddeptoreside,codmcpioreside,dptoreside,sexo,...,personashogar,automovil,computador,internet,lavadora,nivelingles,sociales,ciencias,lectura,global
NaN,1,80,59591,1817,141,92688,542,542,542,135,...,442,451,449,449,474,92932,583775,583775,583775,583775


### Columnas Inexistentes

In [6]:
df.drop(["global","lectura","ciencias","sociales"],axis=1, inplace=True)
df.head()

,periodo,tipodocumento,documento,zona,bilingue,calendario,caracter,daneestablecimiento,danesede,codigodpto,...,edupadre,estrato,personashogar,automovil,computador,internet,lavadora,nivelingles,ingles,mate
0,20132,CC,SB11201320520337,URBANO,N,A,TÉCNICO/ACADÉMICO,1.110010e+11,111001015776,11,...,Secundaria (Bachillerato) completa,Estrato 3,Cinco,No,Si,Si,Si,NaN,40.0,44.0
1,20132,TI,SB11201320545839,URBANO,N,A,ACADÉMICO,1.080010e+11,108001002835,8,...,No sabe,Estrato 1,Cuatro,No,No,No,Si,A1,43.0,28.0
2,20132,TI,SB11201320008819,URBANO,N,A,TÉCNICO/ACADÉMICO,1.730010e+11,173001000359,73,...,Secundaria (Bachillerato) completa,Estrato 2,Cuatro,No,Si,Si,Si,A-,43.0,43.0
3,20132,TI,SB11201320512865,RURAL,N,A,ACADÉMICO,2.707020e+11,270702000120,70,...,Primaria incompleta,Estrato 1,Cinco,No,No,No,No,A-,39.0,34.0
4,20132,TI,SB11201320037607,URBANO,N,A,ACADÉMICO,1.118500e+11,111850001576,11,...,Secundaria (Bachillerato) incompleta,Estrato 2,Seis,No,Si,Si,No,NaN,43.0,34.0


### Completar Nivel de Inglés

In [11]:
df.groupby('nivelingles')['ingles'].describe()

,count,mean,std,min,25%,50%,75%,max
nivelingles,,,,,,,,
A-,275297.0,38.100005,4.710958,0.0,35.0,39.0,42.0,43.0
A1,156946.0,46.262001,2.663476,43.0,45.0,46.0,48.0,52.0
A2,29637.0,56.245706,2.458735,53.0,54.0,56.0,58.0,61.0
B+,9614.0,89.612232,5.558709,83.0,83.0,89.0,94.0,100.0
B1,19349.0,71.309887,5.733368,63.0,66.0,71.0,77.0,80.0


In [14]:
def AsginarNivelIngles(puntaje):
    if puntaje <43:
        return "A-"
    elif puntaje < 53:
        return "A1"
    elif puntaje < 63:
        return "A2"
    elif puntaje < 83:
        return "B1"
    else:
        return "B+"
    
df['nivelingles'] = df['nivelingles'].fillna(df['ingles'].apply(AsginarNivelIngles))

### Crear Nivel de Matemáticas y Corregir Incohernecias

In [15]:
df["mate"].describe()

count    583775.000000
mean         45.074148
std          10.715478
min           0.000000
25%          38.000000
50%          44.000000
75%          51.000000
max         127.000000
Name: mate, dtype: float64

In [16]:
def AsignarNivelMate(puntaje):
    if puntaje < 25:
        return "Bajo"
    elif puntaje <50:
        return "Básico"
    elif puntaje <75:
        return "alto"
    elif puntaje <90:
        return "Superior"
    else:
        return "Excelente"
    
df = df[df["mate"]<= 100]
df["Nivelmate"] = df["mate"].apply(AsignarNivelMate)
df

,periodo,tipodocumento,documento,zona,bilingue,calendario,caracter,daneestablecimiento,danesede,codigodpto,...,estrato,personashogar,automovil,computador,internet,lavadora,nivelingles,ingles,mate,Nivelmate
0,20132,CC,SB11201320520337,URBANO,N,A,TÉCNICO/ACADÉMICO,1.110010e+11,111001015776,11,...,Estrato 3,Cinco,No,Si,Si,Si,A-,40.0,44.0,Básico
1,20132,TI,SB11201320545839,URBANO,N,A,ACADÉMICO,1.080010e+11,108001002835,8,...,Estrato 1,Cuatro,No,No,No,Si,A1,43.0,28.0,Básico
2,20132,TI,SB11201320008819,URBANO,N,A,TÉCNICO/ACADÉMICO,1.730010e+11,173001000359,73,...,Estrato 2,Cuatro,No,Si,Si,Si,A-,43.0,43.0,Básico
3,20132,TI,SB11201320512865,RURAL,N,A,ACADÉMICO,2.707020e+11,270702000120,70,...,Estrato 1,Cinco,No,No,No,No,A-,39.0,34.0,Básico
4,20132,TI,SB11201320037607,URBANO,N,A,ACADÉMICO,1.118500e+11,111850001576,11,...,Estrato 2,Seis,No,Si,Si,No,A1,43.0,34.0,Básico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583770,20132,TI,SB11201320391259,URBANO,N,A,TÉCNICO/ACADÉMICO,1.136570e+11,113657000107,13,...,Estrato 1,Seis,No,No,No,Si,A-,43.0,38.0,Básico
583771,20132,CC,SB11201320239983,RURAL,NaN,A,ACADÉMICO,3.051480e+11,305148000935,5,...,Estrato 4,Cinco,Si,Si,Si,Si,A1,45.0,46.0,Básico
583772,20132,CC,SB11201320277644,URBANO,N,A,ACADÉMICO,3.200010e+11,320001006112,20,...,Estrato 1,Una,No,No,Si,Si,A-,23.0,49.0,Básico
583773,20132,TI,SB11201320454077,URBANO,N,A,ACADÉMICO,1.761090e+11,176109000800,76,...,Estrato 1,Seis,No,No,No,No,B1,71.0,44.0,Básico


### Seleccionar Nacionalidad

In [17]:
df = df[df["nacionalidad"]== "COLOMBIA"]

### Seleccionar Edades Coherentes

In [18]:
df["Edad"] = 2019-(df["fechanac"].str.split("/").str[2]).astype(int)
df = df[(df['Edad'] > 14) & (df['Edad'] < 25)]
df

C:\Users\felip\AppData\Local\Temp\ipykernel_18312\1901518006.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Edad"] = 2019-(df["fechanac"].str.split("/").str[2]).astype(int)


,periodo,tipodocumento,documento,zona,bilingue,calendario,caracter,daneestablecimiento,danesede,codigodpto,...,personashogar,automovil,computador,internet,lavadora,nivelingles,ingles,mate,Nivelmate,Edad
1,20132,TI,SB11201320545839,URBANO,N,A,ACADÉMICO,1.080010e+11,108001002835,8,...,Cuatro,No,No,No,Si,A1,43.0,28.0,Básico,23
2,20132,TI,SB11201320008819,URBANO,N,A,TÉCNICO/ACADÉMICO,1.730010e+11,173001000359,73,...,Cuatro,No,Si,Si,Si,A-,43.0,43.0,Básico,23
3,20132,TI,SB11201320512865,RURAL,N,A,ACADÉMICO,2.707020e+11,270702000120,70,...,Cinco,No,No,No,No,A-,39.0,34.0,Básico,24
4,20132,TI,SB11201320037607,URBANO,N,A,ACADÉMICO,1.118500e+11,111850001576,11,...,Seis,No,Si,Si,No,A1,43.0,34.0,Básico,23
6,20132,TI,SB11201320047502,URBANO,N,A,ACADÉMICO,3.110010e+11,311001036527,11,...,Cuatro,Si,Si,Si,Si,B1,68.0,46.0,Básico,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583768,20132,TI,SB11201320449654,URBANO,N,A,ACADÉMICO,1.130520e+11,113052000172,13,...,Seis,No,No,No,Si,A-,40.0,54.0,alto,22
583769,20132,CC,SB11201320529417,RURAL,N,A,TÉCNICO,2.130060e+11,213006001975,13,...,Seis,No,No,No,Si,A-,42.0,41.0,Básico,24
583770,20132,TI,SB11201320391259,URBANO,N,A,TÉCNICO/ACADÉMICO,1.136570e+11,113657000107,13,...,Seis,No,No,No,Si,A-,43.0,38.0,Básico,23
583773,20132,TI,SB11201320454077,URBANO,N,A,ACADÉMICO,1.761090e+11,176109000800,76,...,Seis,No,No,No,No,B1,71.0,44.0,Básico,24


### Descartar Precidiarios

In [19]:
df.groupby("libertad").describe()

periodo                                                     \
             count          mean       std      min      25%      50%   
libertad                                                                
N         468986.0  20131.950734  0.216423  20131.0  20132.0  20132.0   
S             61.0  20132.000000  0.000000  20132.0  20132.0  20132.0   

                           daneestablecimiento                ...  mate  \
              75%      max               count          mean  ...   75%   
libertad                                                      ...         
N         20132.0  20132.0            468929.0  2.088246e+11  ...  52.0   
S         20132.0  20132.0                61.0  2.628207e+11  ...  46.0   

                     Edad                                                     
            max     count       mean       std   min   25%   50%   75%   max  
libertad                                                                      
N         100.0  468986.0  22.945312  0.772751  16.0  22.0  23.0  23.0  24.0  
S          68.0      61.0  23.540984  0.696988  21.0  23.0  24.0  24.0  24.0  

[2 rows x 104 columns]

In [20]:
df = df[df["libertad"]=="N"]

### Columnas Innecesarias

In [21]:
df.drop(["documento","daneestablecimiento","danesede","codigodpto","codigomcpio","codigoicfes","nombreest",
         "nombresede","sedeprincipal","coddptopres","codmcpiopres","coddeptoreside","codmcpioreside","dptopres",
         "investigacion","tipoestudiante","libertad","mcpiopres","mcpioreside","mcpio", "bilingue","paisreside","nacionalidad"],
        axis=1, inplace=True)
df.head()

C:\Users\felip\AppData\Local\Temp\ipykernel_18312\626980104.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(["documento","daneestablecimiento","danesede","codigodpto","codigomcpio","codigoicfes","nombreest",


,periodo,tipodocumento,zona,calendario,caracter,dpto,genero,jornada,naturaleza,dptoreside,...,personashogar,automovil,computador,internet,lavadora,nivelingles,ingles,mate,Nivelmate,Edad
1,20132,TI,URBANO,A,ACADÉMICO,ATLANTICO,MIXTO,MAÑANA,OFICIAL,ATLANTICO,...,Cuatro,No,No,No,Si,A1,43.0,28.0,Básico,23
2,20132,TI,URBANO,A,TÉCNICO/ACADÉMICO,TOLIMA,MIXTO,COMPLETA,OFICIAL,TOLIMA,...,Cuatro,No,Si,Si,Si,A-,43.0,43.0,Básico,23
3,20132,TI,RURAL,A,ACADÉMICO,SUCRE,MIXTO,TARDE,OFICIAL,SUCRE,...,Cinco,No,No,No,No,A-,39.0,34.0,Básico,24
4,20132,TI,URBANO,A,ACADÉMICO,BOGOTA,MIXTO,TARDE,OFICIAL,BOGOTÁ,...,Seis,No,Si,Si,No,A1,43.0,34.0,Básico,23
6,20132,TI,URBANO,A,ACADÉMICO,BOGOTA,MIXTO,COMPLETA,NO OFICIAL,BOGOTÁ,...,Cuatro,Si,Si,Si,Si,B1,68.0,46.0,Básico,22
